In [1]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

In [27]:
df = pd.read_csv("RFBacks-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,...,3cone,rushatt,rushyds,rushypa,rushtds,hasbling,allamerican,hasawards,heisman,drafted
0,1997,james allen,Oklahoma,RB,70.1,212,NaN,11.0,31.0,NaN,...,NaN,539.0,2387.0,4.4,7.0,0.0,0.0,0.0,0.0,N
1,1997,darnell autry,Northwestern,RB,70.8,210,NaN,NaN,34.0,114.0,...,NaN,787.0,3793.0,4.8,35.0,0.0,0.0,0.0,0.0,Y
2,1997,tiki barber,Virginia,RB,69.4,203,4.56,15.0,35.5,120.0,...,7.18,651.0,3389.0,5.2,31.0,1.0,0.0,1.0,0.0,Y
3,1997,terry battle,Arizona State,RB,70.6,197,4.71,16.0,34.5,118.0,...,NaN,334.0,1796.0,5.4,21.0,0.0,0.0,0.0,0.0,Y
4,1997,calvin branch,Colorado State,RB,71.4,200,4.49,18.0,39.0,127.0,...,7.12,386.0,2106.0,5.5,24.0,0.0,0.0,0.0,0.0,Y


In [9]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [28]:
for index, row in df.iterrows():
    if (df.loc[index,"hasbling"] != 1.0):
        if (df.loc[index,"hasbling"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                try:
                    df.loc[index, "rushatt"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_att']").text
                    df.loc[index, "rushyds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds']").text
                    df.loc[index, "rushypa"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds_per_att']").text
                    df.loc[index, "rushtds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_td']").text
                except:
                    df.loc[index, "rushatt"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_att']").text
                    df.loc[index, "rushyds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds']").text
                    df.loc[index, "rushypa"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds_per_att']").text
                    df.loc[index, "rushtds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_td']").text
                            
                if (len(browser.find_by_css("ul#bling")) > 0):
                    df.loc[index,"hasbling"] = 1
                else:
                    df.loc[index,"hasbling"] = 0
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    df.loc[index,"allamerican"] = 1
                else:
                    df.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("li.bling_hof")) > 0):
                    df.loc[index,"heisman"] = 1
                else:
                    df.loc[index,"heisman"] = 0
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    df.loc[index,"hasawards"] = 1
                else:
                    df.loc[index,"hasawards"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                df.loc[index,"hasbling"] = 0
                df.loc[index,"allamerican"] = 0
                df.loc[index,"heisman"] = 0
                df.loc[index,"hasawards"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            try:
                                df.loc[index, "rushatt"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_att']").text
                                df.loc[index, "rushyds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds']").text
                                df.loc[index, "rushypa"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds_per_att']").text
                                df.loc[index, "rushtds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_td']").text
                            except:
                                df.loc[index, "rushatt"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_att']").text
                                df.loc[index, "rushyds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds']").text
                                df.loc[index, "rushypa"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds_per_att']").text
                                df.loc[index, "rushtds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_td']").text

                            if (len(browser.find_by_css("ul#bling")) > 0):
                                df.loc[index,"hasbling"] = 1
                            else:
                                df.loc[index,"hasbling"] = 0
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                df.loc[index,"allamerican"] = 1
                            else:
                                df.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("li.bling_hof")) > 0):
                                df.loc[index,"heisman"] = 1
                            else:
                                df.loc[index,"heisman"] = 0
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                df.loc[index,"hasawards"] = 1
                            else:
                                df.loc[index,"hasawards"] = 0
                            break
                        else:
                            df.loc[index,"hasbling"] = 0
                            df.loc[index,"allamerican"] = 0
                            df.loc[index,"heisman"] = 0
                            df.loc[index,"hasawards"] = 0
                            break
                    else:
                        df.loc[index,"hasbling"] = 0
                        df.loc[index,"allamerican"] = 0
                        df.loc[index,"heisman"] = 0
                        df.loc[index,"hasawards"] = 0

In [29]:
df.to_csv("RFBacks-Train.csv", index=False)